In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

df_clean = pd.read_pickle("/content/drive/MyDrive/df_clean.pkl")
df_clean.head()


Mounted at /content/drive


,title,description,job_skills,job_industries,pay_period,compensation_type,max_salary,med_salary,min_salary,normalized_salary,...,country_grouped,city,fips,name,description_company,address,follower_count,currency,company_size_label,posting_days
0,Marketing Coordinator,Job descriptionA leading real estate firm in N...,Marketing; Sales,Real Estate,HOURLY,BASE_SALARY,20.0,NaN,17.0,38480.0,...,US,Jersey City,34021.0,Corcoran Sawyer Smith,With years of experience helping local buyers ...,242 Tenth Street,2351.0,USD,50–160 employees,30.0
1,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",Health Care Provider,NaN,HOURLY,BASE_SALARY,50.0,NaN,30.0,83200.0,...,NaN,NaN,8069.0,NaN,NaN,NaN,NaN,USD,NaN,30.0
2,Assitant Restaurant Manager,The National Exemplar is accepting application...,Management; Manufacturing,Restaurants,YEARLY,BASE_SALARY,65000.0,NaN,45000.0,55000.0,...,US,Mariemont,39061.0,The National Exemplar,"In April of 1983, The National Exemplar began ...",6880 Wooster Pike,40.0,USD,10–70 employees,30.0
3,Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,Other,Law Practice,YEARLY,BASE_SALARY,175000.0,NaN,140000.0,157500.0,...,US,Lake Success,36059.0,"Abrams Fensterman, LLP","Abrams Fensterman, LLP is a full-service law f...",3 Dakota Drive,2427.0,USD,50–160 employees,30.0
4,Service Technician,Looking for HVAC service tech with experience ...,Information Technology,Facilities Services,YEARLY,BASE_SALARY,80000.0,NaN,60000.0,70000.0,...,NaN,NaN,19057.0,NaN,NaN,NaN,NaN,USD,NaN,30.0


In [ ]:
# Step 1: Missing summary
missing_count = df_clean.isnull().sum()
missing_percent = (missing_count / len(df_clean)) * 100

missing_table = pd.DataFrame({
    "missing_count": missing_count,
    "missing_percent": missing_percent
}).sort_values("missing_percent", ascending=False)

missing_table


,missing_count,missing_percent
closed_time,122776,99.133622
skills_desc,121410,98.030666
med_salary,119066,96.138039
remote_allowed,108603,87.689848
applies,103426,83.509758
job_benefit_categories,95114,76.798359
max_salary,94420,76.237999
min_salary,94349,76.180672
normalized_salary,88728,71.642080
compensation_type,87776,70.873402


In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 47 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   title                       123849 non-null  object  
 1   description                 123842 non-null  object  
 2   job_skills                  122096 non-null  category
 3   job_industries              122313 non-null  category
 4   pay_period                  36073 non-null   category
 5   compensation_type           36073 non-null   category
 6   max_salary                  29429 non-null   float64 
 7   med_salary                  4783 non-null    float64 
 8   min_salary                  29500 non-null   float64 
 9   normalized_salary           35121 non-null   float64 
 10  job_benefit_categories      28735 non-null   category
 11  views                       105258 non-null  float64 
 12  skills_desc                 2439 non-null    object  
 13 

In [ ]:
# Backup current clean df
df_clean_backup = df_clean.copy()

# Keep only rows where target exists
df_model = df_clean[df_clean['normalized_salary'].notna()].copy()

# Quick report
print("df_clean shape:", df_clean.shape)
print("df_model shape (target not null):", df_model.shape)
print("Rows removed:", df_clean.shape[0] - df_model.shape[0])


df_clean shape: (123849, 47)
df_model shape (target not null): (35121, 47)
Rows removed: 88728


In [ ]:
# Step 1: Missing summary
missing_count = df_model.isnull().sum()
missing_percent = (missing_count / len(df_model)) * 100

missing_table = pd.DataFrame({
    "missing_count": missing_count,
    "missing_percent": missing_percent
}).sort_values("missing_percent", ascending=False)

missing_table

,missing_count,missing_percent
closed_time,34953,99.521654
skills_desc,34794,99.068933
remote_allowed,30451,86.703112
med_salary,30345,86.401298
applies,27681,78.816093
job_benefit_categories,24815,70.655733
posting_domain,14542,41.405427
application_url,13008,37.037670
fips,8400,23.917314
formatted_experience_level,7879,22.433871


In [ ]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35121 entries, 0 to 123848
Data columns (total 47 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   title                       35121 non-null  object  
 1   description                 35120 non-null  object  
 2   job_skills                  34586 non-null  category
 3   job_industries              34735 non-null  category
 4   pay_period                  35121 non-null  category
 5   compensation_type           35121 non-null  category
 6   max_salary                  28893 non-null  float64 
 7   med_salary                  4776 non-null   float64 
 8   min_salary                  28903 non-null  float64 
 9   normalized_salary           35121 non-null  float64 
 10  job_benefit_categories      10306 non-null  category
 11  views                       28281 non-null  float64 
 12  skills_desc                 327 non-null    object  
 13  formatted_work_type 

In [9]:
# df_model.to_csv('df_model.csv', index=False)


In [10]:
# from google.colab import files
# files.download('df_model.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
# Backup before dropping columns
df_model_backup_drop = df_model.copy()

# Columns to remove
cols_to_drop = [
    'compensation_type',
    'work_type',
    'sponsored',
    'job_posting_url',
    'application_url',
    'url',
    'posting_domain'
]

# Drop from df_model
df_model = df_model.drop(columns=cols_to_drop, errors='ignore')

# Check result
print(df_model.shape)
df_model.info()

(35121, 40)
<class 'pandas.core.frame.DataFrame'>
Index: 35121 entries, 0 to 123848
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   title                       35121 non-null  object  
 1   description                 35120 non-null  object  
 2   job_skills                  34586 non-null  category
 3   job_industries              34735 non-null  category
 4   pay_period                  35121 non-null  category
 5   max_salary                  28893 non-null  float64 
 6   med_salary                  4776 non-null   float64 
 7   min_salary                  28903 non-null  float64 
 8   normalized_salary           35121 non-null  float64 
 9   job_benefit_categories      10306 non-null  category
 10  views                       28281 non-null  float64 
 11  skills_desc                 327 non-null    object  
 12  formatted_work_type         35121 non-null  category
 13  remote_a

In [15]:
# STEP: Fill missing job_benefit_categories by company-wise mode (most frequent)
import numpy as np

# Backup before mutation
df_model_backup = df_model.copy()

# Build mapping: for each company_name, take the most frequent non-null benefit string
mode_map = (
    df_model.dropna(subset=['company_name'])
            .groupby('company_name')['job_benefit_categories']
            .apply(lambda s: s.dropna().mode().iloc[0] if not s.dropna().empty else np.nan)
)

# Fill only where job_benefit_categories is missing
before = df_model['job_benefit_categories'].isna().sum()
mask = df_model['job_benefit_categories'].isna() & df_model['company_name'].notna()
df_model.loc[mask, 'job_benefit_categories'] = df_model.loc[mask, 'company_name'].map(mode_map)
after = df_model['job_benefit_categories'].isna().sum()

print(f"Filled {before - after} missing values in 'job_benefit_categories' using company-wise mode.")
print("Active DF is: df_model | Backup: df_model_backup")


Filled 10843 missing values in 'job_benefit_categories' using company-wise mode.
Active DF is: df_model | Backup: df_model_backup


In [17]:
# Validation: count how many values were filled per company
filled_mask = (
    df_model_backup['job_benefit_categories'].isna() &
    df_model['job_benefit_categories'].notna()
)

filled_companies = (
    df_model.loc[filled_mask, 'company_name']
            .value_counts()
            .head(10)  # top 10 companies with most fills
)

filled_companies


,count
company_name,
The Job Network,281
Insight Global,255
Creative Financial Staffing (CFS),132
Vaco,122
Jobot,116
Amazon,111
Robert Half,106
Intermountain Health,93
Apex Systems,83


In [18]:
# Missing before vs. after
before = df_model_backup['job_benefit_categories'].isna().sum()
after = df_model['job_benefit_categories'].isna().sum()

print("Before:", before)
print("After :", after)
print("Filled:", before - after)


Before: 24815
After : 13972
Filled: 10843


In [19]:
import numpy as np

# Backup before this fill step
df_model_backup_industry = df_model.copy()

# Build industry-wise mode map
industry_mode_map = (
    df_model.dropna(subset=['industry_group_15'])
            .groupby('industry_group_15')['job_benefit_categories']
            .apply(lambda s: s.dropna().mode().iloc[0] if not s.dropna().empty else np.nan)
)

# Mask: missing job_benefit_categories AND industry_group_15 exists
mask_ind = (
    df_model['job_benefit_categories'].isna() &
    df_model['industry_group_15'].notna()
)

before = df_model['job_benefit_categories'].isna().sum()

# Fill from industry mode
df_model.loc[mask_ind, 'job_benefit_categories'] = (
    df_model.loc[mask_ind, 'industry_group_15'].map(industry_mode_map)
)

after = df_model['job_benefit_categories'].isna().sum()

print(f"Filled {before - after} additional values using industry_group_15 mode.")
print("Active DF: df_model | Backup created: df_model_backup_industry")


Filled 13550 additional values using industry_group_15 mode.
Active DF: df_model | Backup created: df_model_backup_industry


/tmp/ipython-input-504217131.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('industry_group_15')['job_benefit_categories']


In [20]:
# Identify which rows were filled in this step
filled_mask_ind = (
    df_model_backup_industry['job_benefit_categories'].isna() &
    df_model['job_benefit_categories'].notna()
)

# Count fills per industry (top 10)
filled_by_industry = (
    df_model.loc[filled_mask_ind, 'industry_group_15']
            .value_counts()
            .head(10)
)

filled_by_industry


,count
industry_group_15,
Professional & Business Services,3279
Healthcare & Life Sciences,1606
IT Services,1305
Tech & Internet,1175
Finance & Insurance,1111
Manufacturing,826
Retail & Wholesale,813
Other,626
Education & Research,604


In [21]:
before_ind = df_model_backup_industry['job_benefit_categories'].isna().sum()
after_ind  = df_model['job_benefit_categories'].isna().sum()

print("Before (industry fill):", before_ind)
print("After  (industry fill):", after_ind)
print("Filled:", before_ind - after_ind)


Before (industry fill): 13972
After  (industry fill): 422
Filled: 13550


In [23]:
# Backup before dropping columns
df_model_backup_drop2 = df_model.copy()

cols_to_drop_2 = ['closed_time', 'skills_desc']

df_model = df_model.drop(columns=cols_to_drop_2, errors='ignore')

df_model.shape


(35121, 38)

In [30]:
# === Compute industry midpoint (mean of (min+max)/2) ===
valid_minmax = df_model[["industry_group_15","min_salary","max_salary"]].dropna()
midpoint_by_ind = (
    ((valid_minmax["min_salary"] + valid_minmax["max_salary"]) / 2)
    .groupby(valid_minmax["industry_group_15"])
    .mean()
)

# Fill only where med_salary is NA and industry exists in map
ms_missing = df_model["med_salary"].isna()
can_fill = ms_missing & df_model["industry_group_15"].isin(midpoint_by_ind.index)
df_model.loc[can_fill, "med_salary"] = df_model.loc[can_fill, "industry_group_15"].map(midpoint_by_ind)

print("Filled med_salary by industry midpoint:", int(can_fill.sum()))
print("Still missing med_salary:", int(df_model["med_salary"].isna().sum()))


Filled med_salary by industry midpoint: 29836
Still missing med_salary: 509


In [31]:
# Identify rows that were filled in this step
filled_mask_med = (
    df_model_backup_med['med_salary'].isna() &
    df_model['med_salary'].notna()
)

filled_mask_med.sum()


np.int64(29836)

In [32]:
# Count how many med_salary values were filled per industry
filled_by_industry = (
    df_model.loc[filled_mask_med, 'industry_group_15']
            .value_counts()
            .head(10)
)

filled_by_industry


,count
industry_group_15,
Professional & Business Services,9040
Healthcare & Life Sciences,3676
IT Services,2534
Tech & Internet,2459
Finance & Insurance,2426
Manufacturing,1864
Retail & Wholesale,1704
Other,1076
Construction & Real Estate,976


In [33]:
before_med = df_model_backup_med['med_salary'].isna().sum()
after_med  = df_model['med_salary'].isna().sum()

print("Before:", before_med)
print("After :", after_med)
print("Filled:", before_med - after_med)


Before: 30345
After : 509
Filled: 29836


In [41]:
# Distribution of remote_allowed by industry
pd.crosstab(
    df_model['industry_group_15'],
    df_model['remote_allowed'],
    normalize='index'   # percentage per industry
).round(3)

pd.crosstab(
    df_model['industry_group_15'],
    df_model['remote_allowed']
)



remote_allowed,1.0
industry_group_15,
Agriculture & Natural Resources,23
Construction & Real Estate,59
Education & Research,110
"Energy, Utilities & Mining",22
Finance & Insurance,452
Government & Public Sector,36
Healthcare & Life Sciences,522
"Hospitality, Travel & Leisure",42
IT Services,607


In [38]:
pd.crosstab(
    df_model['state_region_5'],
    df_model['remote_allowed']
)


remote_allowed,1.0
state_region_5,
International,132
Midwest,584
Northeast,820
Other,652
South,1129
West,1247


In [44]:
# STEP: make a dummy for remote_allowed (missing->0, present->1) and keep original
df_model_bak = df_model.copy()  # backup per guideline

# 0 if NaN, 1 if any value present (category-safe)
df_model['remote_allowed_dummy'] = df_model['remote_allowed'].notna().astype('int')

# quick sanity check
df_model['remote_allowed_dummy'].value_counts(dropna=False)


,count
remote_allowed_dummy,
0,30451
1,4670


In [46]:
# Backup before dropping columns
df_model_backup_drop3 = df_model.copy()

cols_to_drop_3 = ['remote_allowed']

df_model = df_model.drop(columns=cols_to_drop_3, errors='ignore')

df_model.shape

(35121, 38)

In [56]:
# Backup before dropping columns
df_model_backup_drop4 = df_model.copy()

cols_to_drop_4 = ['applies']

df_model = df_model.drop(columns=cols_to_drop_4, errors='ignore')

df_model.shape

(35121, 37)

In [49]:
pd.crosstab(
    df_model['formatted_work_type'],
    df_model['formatted_experience_level'],
    normalize='index'
).round(3)


formatted_experience_level,Associate,Director,Entry level,Executive,Internship,Mid-Senior level
formatted_work_type,,,,,,
Contract,0.247,0.012,0.193,0.002,0.001,0.545
Full-time,0.134,0.049,0.333,0.011,0.009,0.464
Internship,0.006,0.011,0.144,0.000,0.753,0.086
Other,0.000,0.000,0.621,0.026,0.017,0.336
Part-time,0.044,0.005,0.618,0.004,0.021,0.308
Temporary,0.202,0.015,0.301,0.009,0.009,0.464
Volunteer,0.667,0.000,0.333,0.000,0.000,0.000


In [50]:
# Backup
df_model_backup_exp = df_model.copy()

# Standardize text to avoid case/spacing issues
df_model['formatted_work_type'] = df_model['formatted_work_type'].astype('string').str.strip().str.lower()
df_model['formatted_experience_level'] = df_model['formatted_experience_level'].astype('string').str.strip()

# Mask: missing experience BUT clearly internship by work_type
mask_int = (
    df_model['formatted_experience_level'].isna() &
    (df_model['formatted_work_type'] == 'internship')
)

# Fill
df_model.loc[mask_int, 'formatted_experience_level'] = 'Internship'

print("Filled:", mask_int.sum())
print("Remaining missing in formatted_experience_level:",
      df_model['formatted_experience_level'].isna().sum())


Filled: 65
Remaining missing in formatted_experience_level: 7814


In [58]:
pd.crosstab(
    df_model['formatted_work_type'],
    df_model['formatted_experience_level'],
    normalize='index'
).round(3)


formatted_experience_level,Associate,Director,Entry level,Executive,Internship,Mid-Senior level
formatted_work_type,,,,,,
contract,0.247,0.012,0.193,0.002,0.001,0.545
full-time,0.134,0.049,0.333,0.011,0.009,0.464
internship,0.004,0.008,0.105,0.000,0.820,0.063
other,0.000,0.000,0.621,0.026,0.017,0.336
part-time,0.044,0.005,0.618,0.004,0.021,0.308
temporary,0.202,0.015,0.301,0.009,0.009,0.464
volunteer,0.667,0.000,0.333,0.000,0.000,0.000


In [62]:
pd.crosstab(
    [df_model['formatted_work_type'], df_model['industry_group_15']],
    df_model['formatted_experience_level'],
    normalize='index'
).round(3
)


formatted_experience_level                            Associate  Director  \
formatted_work_type industry_group_15                                       
contract            Construction & Real Estate            0.154     0.077   
                    Education & Research                  0.043     0.043   
                    Energy, Utilities & Mining            0.429     0.000   
                    Finance & Insurance                   0.150     0.050   
                    Government & Public Sector            0.000     0.000   
...                                                         ...       ...   
temporary           Professional & Business Services      0.342     0.007   
                    Retail & Wholesale                    0.000     0.000   
                    Tech & Internet                       0.333     0.000   
                    Transport & Logistics                 0.000     0.000   
volunteer           Other                                 0.667     0.000   

formatted_experience_level                            Entry level  Executive  \
formatted_work_type industry_group_15                                          
contract            Construction & Real Estate              0.231       0.00   
                    Education & Research                    0.565       0.00   
                    Energy, Utilities & Mining              0.143       0.00   
                    Finance & Insurance                     0.250       0.00   
                    Government & Public Sector              0.250       0.00   
...                                                           ...        ...   
temporary           Professional & Business Services        0.204       0.02   
                    Retail & Wholesale                      0.529       0.00   
                    Tech & Internet                         0.000       0.00   
                    Transport & Logistics                   0.500       0.00   
volunteer           Other                                   0.333       0.00   

formatted_experience_level                            Internship  \
formatted_work_type industry_group_15                              
contract            Construction & Real Estate               0.0   
                    Education & Research                     0.0   
                    Energy, Utilities & Mining               0.0   
                    Finance & Insurance                      0.0   
                    Government & Public Sector               0.0   
...                                                          ...   
temporary           Professional & Business Services         0.0   
                    Retail & Wholesale                       0.0   
                    Tech & Internet                          0.0   
                    Transport & Logistics                    0.0   
volunteer           Other                                    0.0   

formatted_experience_level                            Mid-Senior level  
formatted_work_type industry_group_15                                   
contract            Construction & Real Estate                   0.538  
                    Education & Research                         0.348  
                    Energy, Utilities & Mining                   0.429  
                    Finance & Insurance                          0.550  
                    Government & Public Sector                   0.750  
...                                                                ...  
temporary           Professional & Business Services             0.428  
                    Retail & Wholesale                           0.471  
                    Tech & Internet                              0.667  
                    Transport & Logistics                        0.500  
volunteer           Other                                        0.000  

[85 rows x 6 columns]

In [63]:
import numpy as np
import pandas as pd

# 1) Work on df_model; avoid categorical fill errors
df_model['formatted_experience_level'] = df_model['formatted_experience_level'].astype('object')

# 2) Build (work_type, industry) → mode(experience_level) table
mode_map = (
    df_model
    .groupby(['formatted_work_type', 'industry_group_15'])['formatted_experience_level']
    .agg(lambda s: s.dropna().mode().iloc[0] if not s.dropna().mode().empty else np.nan)
    .rename('fel_mode')
    .reset_index()
)

# 3) Merge and fill only the missing
before_na = df_model['formatted_experience_level'].isna().sum()
df_model = df_model.merge(mode_map, on=['formatted_work_type', 'industry_group_15'], how='left')

mask = df_model['formatted_experience_level'].isna() & df_model['fel_mode'].notna()
df_model.loc[mask, 'formatted_experience_level'] = df_model.loc[mask, 'fel_mode']

# 4) Cleanup and (optionally) cast back to category
df_model.drop(columns=['fel_mode'], inplace=True)
df_model['formatted_experience_level'] = df_model['formatted_experience_level'].astype('category')

print(f"Imputed experience_level rows: {mask.sum()} (from {before_na} missing)")


Imputed experience_level rows: 7379 (from 7814 missing)


In [66]:
# Remaining NA count
df_model['formatted_experience_level'].isna().sum()


np.int64(435)

In [80]:
missing_count = df_model.isnull().sum()
missing_percent = (missing_count / len(df_model)) * 100

missing_table = pd.DataFrame({
    "missing_count": missing_count,
    "missing_percent": missing_percent
}).sort_values("missing_percent", ascending=False)

missing_table

,missing_count,missing_percent
fips,8400,23.917314
views,6840,19.475527
follower_count,6626,18.866205
zip_code,6611,18.823496
max_salary,6228,17.732980
min_salary,6218,17.704507
posting_days,5727,16.306483
company_size_label,2169,6.175792
expiry,1903,5.418411
expiry_month,1903,5.418411


In [76]:
# Backup before conversion
df_model_backup_cat = df_model.copy()

cols_to_cat = [
    'company_name',
    'industry_group_15',
    'formatted_work_type',
    'job_benefit_categories'
]

for c in cols_to_cat:
    df_model[c] = df_model[c].astype('category')

# Check result
df_model[cols_to_cat].dtypes


,0
company_name,category
industry_group_15,category
formatted_work_type,category
job_benefit_categories,category


In [78]:
df_model['remote_allowed_dummy'] = df_model['remote_allowed_dummy'].astype(float)
df_model['remote_allowed_dummy'].dtype


dtype('float64')

In [79]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35121 entries, 0 to 35120
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   title                       35121 non-null  object  
 1   description                 35120 non-null  object  
 2   job_skills                  34586 non-null  category
 3   job_industries              34735 non-null  category
 4   pay_period                  35121 non-null  category
 5   max_salary                  28893 non-null  float64 
 6   med_salary                  34612 non-null  float64 
 7   min_salary                  28903 non-null  float64 
 8   normalized_salary           35121 non-null  float64 
 9   job_benefit_categories      34699 non-null  category
 10  views                       28281 non-null  float64 
 11  formatted_work_type         35121 non-null  category
 12  application_type            35121 non-null  category
 13  formatted_experi

In [81]:
# === BLOCK A: fill simple categorical/object NAs with "Unknown" ===
# English comments only.

# columns to fill with a plain "Unknown"
unknown_cols = [
    "job_skills",
    "job_industries",
    "description_company",
    "address",
    "name",
    "company_name",
    "country_grouped",
]

def fill_unknown_safe(df, cols):
    for c in cols:
        if c not in df.columns:
            continue
        s = df[c]
        if pd.api.types.is_categorical_dtype(s):
            # add category first, then fill
            df[c] = s.cat.add_categories(["Unknown"]).fillna("Unknown")
        else:
            df[c] = s.fillna("Unknown")
    return df

# apply
df_model = fill_unknown_safe(df_model, unknown_cols)

# quick check that these columns have no remaining NAs
check = df_model[unknown_cols].isna().sum().sort_values(ascending=False)
print("Remaining NAs (Block A targets):")
print(check[check > 0])


Remaining NAs (Block A targets):
Series([], dtype: int64)


/tmp/ipython-input-67706264.py:20: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(s):


In [84]:
# === BLOCK B: fill categorical missing values via MODE ===

import pandas as pd
from pandas.api.types import is_categorical_dtype

# ---- Patch for categorical mode imputation (safe) ----
def fillna_with_mode_safe(s: pd.Series) -> pd.Series:
    # compute mode (fallback to "Unknown" if no mode)
    mode_vals = s.mode(dropna=True)
    mode_val = mode_vals.iloc[0] if not mode_vals.empty else "Unknown"

    if is_categorical_dtype(s):
        # add category only if missing
        if mode_val not in s.cat.categories:
            s = s.cat.add_categories([mode_val])
        s = s.fillna(mode_val)
        # optional: remove unused cats (keeps dtype tidy)
        s = s.cat.remove_unused_categories()
        return s
    else:
        return s.fillna(mode_val)

# example: re-run for your Block A columns
block_a_cols = [
    "company_size_label", "industry_group_15", "job_skills", "state_region_5",
    "city", "company_name", "country_grouped", "formatted_experience_level",
    "job_benefit_categories", "job_industries"
]

for c in block_a_cols:
    df_model[c] = fillna_with_mode_safe(df_model[c])

# quick check
print("Remaining NAs (Block A targets):")
print(df_model[block_a_cols].isna().sum()[df_model[block_a_cols].isna().sum() > 0])



Remaining NAs (Block A targets):
Series([], dtype: int64)


/tmp/ipython-input-1763630970.py:12: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if is_categorical_dtype(s):


In [85]:
# BLOCK C — numeric median imputation (simple)
# Backup before changes
df_model_backup_C = df_model.copy()

# Columns to fill with column median
num_cols_median = [
    'fips', 'views', 'follower_count', 'zip_code', 'posting_days'
]

import numpy as np
import pandas as pd

filled_counts = {}

for c in num_cols_median:
    if c in df_model.columns:
        n_before = df_model[c].isna().sum()
        if n_before > 0:
            med = df_model[c].median()
            df_model[c] = df_model[c].fillna(med).astype(float)  # keep float
        filled_counts[c] = n_before

print("Filled (count per column):")
print(pd.Series(filled_counts).sort_values(ascending=False))

# Verification for this block only
remaining = df_model[num_cols_median].isna().sum()
print("\nRemaining NAs (Block C targets):")
print(remaining[remaining > 0])


Filled (count per column):
fips              8400
views             6840
follower_count    6626
zip_code          6611
posting_days      5727
dtype: int64

Remaining NAs (Block C targets):
Series([], dtype: int64)


In [86]:
from pandas.api.types import CategoricalDtype
isinstance(df_model[c].dtype, CategoricalDtype)


False

In [87]:
from pandas.api.types import CategoricalDtype
if isinstance(df_model[c].dtype, CategoricalDtype):
    cats = df_model[c].cat.categories
    add = [mode_val] if mode_val not in cats else []
    if add:
        df_model[c] = df_model[c].cat.add_categories(add)
    df_model[c] = df_model[c].fillna(mode_val)


In [89]:
# --- Block C: robust group-median imputation for numeric columns ---
import pandas as pd
import numpy as np

def fill_numeric_by_group_median_safe(df, cols, group_col):
    """Fill NA in numeric cols by group median (group_col), then global median.
    Uses transform to align index and avoids dtype issues."""
    for c in cols:
        # force numeric (per course rules) and keep as float
        s = pd.to_numeric(df[c], errors="coerce").astype("float64")

        # per-row group median (aligned index; robust to categories)
        g_med = s.groupby(df[group_col], observed=False).transform("median")

        # fill by group, then global
        filled = s.fillna(g_med).fillna(s.median())

        df[c] = filled.astype("float64")
    return df

# >>> run on your existing groups of numeric columns (same names you used before)
df_model = fill_numeric_by_group_median_safe(df_model, behavior_cols,   "industry_group_15")
df_model = fill_numeric_by_group_median_safe(df_model, timing_cols,     "industry_group_15")
df_model = fill_numeric_by_group_median_safe(df_model, salary_cols,     "industry_group_15")
df_model = fill_numeric_by_group_median_safe(df_model, geo_code_cols,   "industry_group_15")

# quick check: remaining NAs in the target sets
targets = list(set(behavior_cols + timing_cols + salary_cols + geo_code_cols))
rem_na = df_model[targets].isna().sum()
print("Remaining NAs (Block C targets):")
print(rem_na[rem_na > 0].sort_values(ascending=False))


Remaining NAs (Block C targets):
Series([], dtype: int64)


In [90]:
# Block D — simple fills for low-missing columns

import pandas as pd
import numpy as np

# 1) Backup
df_model_bakD = df_model.copy()  # backup before this block

# 2) Targets (use only those that exist & still have NA)
obj_cols = [c for c in ["description_company", "address", "name", "description"]
            if c in df_model.columns and df_model[c].isna().any()]

num_singleton_cols = [c for c in ["listed_time", "listed_month"]
                      if c in df_model.columns and df_model[c].isna().any()]

# 3) Impute
# 3a) objects → "Unknown"
for c in obj_cols:
    df_model[c] = df_model[c].fillna("Unknown")

# 3b) singleton numerics → median
for c in num_singleton_cols:
    df_model[c] = pd.to_numeric(df_model[c], errors="coerce")
    df_model[c] = df_model[c].fillna(df_model[c].median())

# 4) Check this block's targets
check_cols = obj_cols + num_singleton_cols
print("Remaining NAs (Block D targets):")
print(df_model[check_cols].isna().sum().sort_values(ascending=False) if check_cols else "— None —")


Remaining NAs (Block D targets):
description    0
dtype: int64


In [91]:
# BLOCK E: Final categorical imputation → fill NAs with "Unknown"
# Notes:
# - Only for categorical / object columns (your policy). Numerics stay as-is.
# - If dtype is Categorical, add "Unknown" to categories before fillna.

import pandas as pd

# choose the categorical columns from your latest table
cat_cols_E = [
    "company_size_label", "industry_group_15", "job_skills", "state_region_5",
    "city", "company_name", "country_grouped", "formatted_experience_level",
    "job_benefit_categories", "job_industries", "address", "name",
    "description_company"
]

# keep a backup before mutation
df_model_backup_E = df_model.copy()

def fill_cats_unknown(df, cols, token="Unknown"):
    for c in cols:
        if c not in df.columns:
            continue
        s = df[c]
        # handle pandas Categorical separately
        if isinstance(s.dtype, pd.CategoricalDtype):
            if token not in s.cat.categories:
                s = s.cat.add_categories([token])
            df[c] = s.fillna(token)
        else:
            df[c] = s.fillna(token)
    return df

df_model = fill_cats_unknown(df_model, cat_cols_E, token="Unknown")

# quick validation – show any remaining NAs in these targets
remaining_cats_na = df_model[cat_cols_E].isna().sum()
print("Remaining NAs (Block E categorical targets):")
print(remaining_cats_na[remaining_cats_na > 0])


Remaining NAs (Block E categorical targets):
Series([], dtype: int64)


In [92]:
df_model.isna().sum().sort_values(ascending=False)[lambda x: x > 0]


,0
med_salary,509


In [94]:
# === Block F: Impute med_salary ===

import numpy as np
import pandas as pd

# Backup
df_model_backup_med = df_model.copy()

# Missing BEFORE
print("Missing med_salary BEFORE:", df_model["med_salary"].isna().sum())

# Ensure numeric
df_model["med_salary"] = pd.to_numeric(df_model["med_salary"], errors="coerce")
df_model["max_salary"] = pd.to_numeric(df_model["max_salary"], errors="coerce")
df_model["min_salary"] = pd.to_numeric(df_model["min_salary"], errors="coerce")

# Step 1: (max + min) / 2 where both exist
candidate = ((df_model["max_salary"] + df_model["min_salary"]) / 2).where(
    df_model["max_salary"].notna() & df_model["min_salary"].notna()
)

new_med = df_model["med_salary"].copy()
new_med = new_med.fillna(candidate)

# Step 2: industry median (per-row transform)
industry_meds = df_model.groupby("industry_group_15")["med_salary"].transform("median")
new_med = new_med.fillna(industry_meds)

# Step 3: global fallback
global_med = df_model["med_salary"].median()
new_med = new_med.fillna(global_med)

df_model["med_salary"] = new_med.astype(float)

# Missing AFTER
print("Missing med_salary AFTER:", df_model["med_salary"].isna().sum())


Missing med_salary BEFORE: 509
Missing med_salary AFTER: 0


/tmp/ipython-input-4187629113.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  industry_meds = df_model.groupby("industry_group_15")["med_salary"].transform("median")


In [95]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35121 entries, 0 to 35120
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   title                       35121 non-null  object  
 1   description                 35121 non-null  object  
 2   job_skills                  35121 non-null  category
 3   job_industries              35121 non-null  category
 4   pay_period                  35121 non-null  category
 5   max_salary                  35121 non-null  float64 
 6   med_salary                  35121 non-null  float64 
 7   min_salary                  35121 non-null  float64 
 8   normalized_salary           35121 non-null  float64 
 9   job_benefit_categories      35121 non-null  category
 10  views                       35121 non-null  float64 
 11  formatted_work_type         35121 non-null  category
 12  application_type            35121 non-null  category
 13  formatted_experi

In [96]:
# import pickle

# with open("df_model.pkl", "wb") as f:
#     pickle.dump(df_model, f)

# print("Saved df_model.pkl in notebook directory.")


Saved df_model.pkl in notebook directory.


In [97]:
# from google.colab import drive
# drive.mount('/content/drive')

# with open('/content/drive/MyDrive/df_model.pkl', 'wb') as f:
#     pickle.dump(df_model, f)

# print("Saved df_model.pkl to Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved df_model.pkl to Google Drive.


In [98]:
# from google.colab import files
# files.download("df_model.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>